In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model_108")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=50,
      num_epochs=None,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=3000,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=5,
      shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model_108', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000026C518E9278>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INF

INFO:tensorflow:global_step/sec: 6.84967
INFO:tensorflow:probabilities = [[0.10184409 0.1014573  0.10308447 0.11124679 0.08873885 0.09655334
  0.10523229 0.09581027 0.09438536 0.10164724]
 [0.10137299 0.10032039 0.09468617 0.10099476 0.09410142 0.10165936
  0.11068944 0.09508692 0.10137396 0.09971464]
 [0.10686252 0.09539963 0.0978843  0.10111603 0.08613699 0.08818888
  0.0997282  0.09943023 0.10815506 0.11709826]
 [0.10183688 0.10334429 0.08798283 0.10450644 0.08774221 0.10145113
  0.09807925 0.09601602 0.10797345 0.11106753]
 [0.10702839 0.10290007 0.09564325 0.10235194 0.09275734 0.09608966
  0.10821854 0.09000816 0.09143352 0.11356905]
 [0.09137516 0.0993555  0.09132263 0.11363551 0.08693469 0.09328752
  0.11679646 0.10222818 0.09830146 0.1067629 ]
 [0.09976823 0.09523229 0.09581761 0.10732811 0.08720617 0.10464334
  0.09938831 0.09935865 0.1045439  0.1067134 ]
 [0.10227085 0.1030619  0.09295072 0.11378543 0.09585052 0.09942554
  0.09436844 0.0981294  0.10412705 0.09603017]
 [0.110

INFO:tensorflow:global_step/sec: 6.74326
INFO:tensorflow:probabilities = [[0.10078686 0.0973217  0.09785032 0.10168554 0.08972484 0.08953673
  0.10348733 0.09668527 0.11160816 0.11131321]
 [0.10885882 0.10173109 0.09902121 0.11568117 0.08740229 0.08268284
  0.11446282 0.08896378 0.10430846 0.09688756]
 [0.10448769 0.09284136 0.1023206  0.10603267 0.09474139 0.09317392
  0.10850958 0.09071317 0.10565971 0.10151997]
 [0.11227614 0.09793959 0.09050828 0.10217968 0.08602718 0.0952925
  0.09834281 0.09409861 0.10150175 0.12183349]
 [0.10047619 0.1088125  0.09368635 0.09926236 0.08902411 0.09555113
  0.10746101 0.10032603 0.09950221 0.10589802]
 [0.11455927 0.10084782 0.09778587 0.09916842 0.08068725 0.09848856
  0.11330246 0.09272052 0.10989448 0.09254546]
 [0.10515251 0.10107079 0.09865551 0.1008759  0.08507259 0.09492345
  0.10486945 0.09898505 0.10029594 0.11009882]
 [0.10579355 0.10777549 0.09322882 0.10419542 0.0944869  0.0858371
  0.10360027 0.0908405  0.10282159 0.11142035]
 [0.10449

INFO:tensorflow:global_step/sec: 6.12942
INFO:tensorflow:probabilities = [[0.09882748 0.0950927  0.08953448 0.10389484 0.09785453 0.09234194
  0.10266764 0.09673243 0.10961854 0.11343528]
 [0.09980831 0.08797247 0.10307483 0.09995989 0.0919236  0.09810054
  0.09968026 0.0938924  0.11605014 0.10953768]
 [0.10829488 0.0972329  0.1110623  0.09791107 0.08477113 0.08452605
  0.10995667 0.10008042 0.10815559 0.09800891]
 [0.09762565 0.1007084  0.09950902 0.104019   0.09538206 0.09117253
  0.11036692 0.09307578 0.09828109 0.10985955]
 [0.09853161 0.08781525 0.09531445 0.10328602 0.09467694 0.09501055
  0.10386368 0.10930526 0.10373908 0.10845713]
 [0.10792109 0.09212357 0.10841817 0.11021066 0.08176659 0.10028089
  0.11573313 0.09011933 0.09539899 0.09802758]
 [0.10135075 0.09328327 0.09027205 0.09944244 0.09284674 0.11451291
  0.10328732 0.08677739 0.10555042 0.11267669]
 [0.09793033 0.0994955  0.09243207 0.1117738  0.09373519 0.09398812
  0.10033643 0.0928992  0.10110453 0.11630488]
 [0.103

INFO:tensorflow:global_step/sec: 7.04
INFO:tensorflow:probabilities = [[0.10675503 0.1033764  0.11095121 0.10546116 0.0818188  0.09394694
  0.10710765 0.09205116 0.10265142 0.09588026]
 [0.10882594 0.09368595 0.09467568 0.09966175 0.09354791 0.09839062
  0.10175035 0.09288108 0.10975178 0.10682894]
 [0.11900466 0.09008827 0.10109414 0.10972457 0.09057925 0.10104325
  0.08907585 0.09760439 0.09934424 0.10244145]
 [0.10883471 0.0949496  0.09752979 0.10282308 0.09358795 0.10590559
  0.10243708 0.0889653  0.09620522 0.10876179]
 [0.10311169 0.09803198 0.10565294 0.10721259 0.08512847 0.09420976
  0.10643984 0.09881879 0.10066598 0.10072798]
 [0.10190724 0.08983647 0.09588341 0.1007209  0.10159301 0.09859258
  0.10542913 0.09592837 0.10050256 0.10960627]
 [0.11835479 0.09648626 0.11330462 0.11483901 0.08070743 0.09177481
  0.10825992 0.07814498 0.10384186 0.0942863 ]
 [0.09642686 0.09927157 0.09961446 0.1043116  0.08288868 0.10327632
  0.11342403 0.09291063 0.09927306 0.10860284]
 [0.103969

...

INFO:tensorflow:global_step/sec: 7.20255
INFO:tensorflow:probabilities = [[0.00260061 0.01641256 0.78892565 0.0975778  0.00132465 0.00270949
  0.02020933 0.01811504 0.05072991 0.00139489]
 [0.00009246 0.00015837 0.00001604 0.00220538 0.00761065 0.0011768
  0.00000717 0.9063851  0.00864551 0.07370257]
 [0.06443911 0.00535443 0.05523383 0.0121901  0.20255771 0.2235189
  0.2999523  0.01070353 0.09550761 0.03054253]
 [0.00488647 0.00044637 0.00122226 0.00788505 0.34926894 0.00744129
  0.00064442 0.10342414 0.05557499 0.4692061 ]
 [0.8668107  0.0003716  0.01733238 0.06197343 0.00060404 0.00865684
  0.00515944 0.00463399 0.02695056 0.00750692]
 [0.00011962 0.00482497 0.00284444 0.01694222 0.00065909 0.00047944
  0.00003249 0.93540114 0.02605192 0.01264467]
 [0.00124836 0.00018774 0.00012027 0.00474801 0.06847679 0.01595263
  0.00083294 0.14256668 0.02044102 0.7454256 ]
 [0.07442475 0.17693797 0.08428977 0.07204704 0.06550509 0.19719279
  0.03186102 0.02729291 0.2399728  0.03047586]
 [0.06250

INFO:tensorflow:global_step/sec: 7.21724
INFO:tensorflow:probabilities = [[0.0002327  0.97071236 0.0058868  0.00376604 0.00025097 0.00077748
  0.00288726 0.00091182 0.0129851  0.00158939]
 [0.00459426 0.00024714 0.01157769 0.00190008 0.26078176 0.00118811
  0.00756132 0.02289982 0.02139889 0.6678509 ]
 [0.00125671 0.00297967 0.00021084 0.00157242 0.36594245 0.00259769
  0.00238595 0.0068844  0.00633124 0.6098386 ]
 [0.0001032  0.00128337 0.00014629 0.02357361 0.02243808 0.03452709
  0.00025028 0.20216548 0.02750748 0.68800515]
 [0.00000143 0.00000101 0.00000013 0.00007584 0.00071969 0.00007315
  0.00000005 0.9930622  0.00005407 0.00601244]
 [0.02495394 0.00413535 0.00958717 0.00733534 0.6602657  0.01076741
  0.05032403 0.00135528 0.00727032 0.22400539]
 [0.00150007 0.00003844 0.00006951 0.00774152 0.01287    0.00242096
  0.00000404 0.48230922 0.00750817 0.48553813]
 [0.00009755 0.8930189  0.05755526 0.00258547 0.00757896 0.00190806
  0.00661226 0.00303682 0.020804   0.0068028 ]
 [0.000

INFO:tensorflow:Saving checkpoints for 3000 into /tmp/mnist_convnet_model_108\model.ckpt.
INFO:tensorflow:Loss for final step: 0.43120623.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-04-16-17:35:08
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model_108\model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-04-16-17:35:52
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.8937, global_step = 3000, loss = 0.4138385
{'accuracy': 0.8937, 'loss': 0.4138385, 'global_step': 3000}


SystemExit: 

c:\users\ninad\anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
